# Store Item Demand Forecasting - Model Comparison

このノートブックでは、アルゴリズム（特徴量生成・学習）を `src/` 配下のモジュールから読み込み、可視化と評価に専念します。

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

# srcディレクトリをパスに追加
sys.path.append("/work/competitions/demand-forecasting-kernels-only/")

from src.features import create_date_features, create_lag_features, create_rolling_features
from src.models import train_lgbm, smape

%matplotlib inline
sns.set(style="whitegrid")

In [ ]:
DATA_DIR = "/work/competitions/demand-forecasting-kernels-only/data"
train = pd.read_csv(f"{DATA_DIR}/train.csv", parse_dates=["date"])

# src.features を使用して特徴量生成
train = create_date_features(train)
train = create_lag_features(train, lags=[91, 98, 105, 112])
train = create_rolling_features(train, windows=[30], shift=91)
train = train.dropna()

print(f"Data prepared: {train.shape}")

In [ ]:
split_date = "2017-10-01"
train_data = train[train["date"] < split_date]
val_data = train[train["date"] >= split_date]

In [ ]:
cols_base = ["store", "item", "dayofweek", "month", "sales_lag_91", "sales_lag_98", "sales_lag_105", "sales_lag_112", "rolling_mean_30"]
cols_cyc = ["store", "item", "dayofweek_sin", "dayofweek_cos", "month", "sales_lag_91", "sales_lag_98", "sales_lag_105", "sales_lag_112", "rolling_mean_30"]

print("Training Baseline Model...")
m_base = train_lgbm(train_data[cols_base], train_data["sales"], val_data[cols_base], val_data["sales"])

print("\nTraining Cyclic Model...")
m_cyc = train_lgbm(train_data[cols_cyc], train_data["sales"], val_data[cols_cyc], val_data["sales"])

score_base = smape(m_base.predict(val_data[cols_base]), val_data["sales"])
score_cyc = smape(m_cyc.predict(val_data[cols_cyc]), val_data["sales"])

print(f"\nBaseline SMAPE: {score_base:.6f}")
print(f"Cyclic SMAPE:   {score_cyc:.6f}")

## 特徴量重要度の可視化

この棒グラフは、モデルが予測のためにどの特徴量を重視したかを示しています。

In [ ]:
lgb.plot_importance(m_cyc, importance_type="split", figsize=(10, 8))
plt.title("Feature Importance (Cyclic Model)")
plt.show()